In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lin = nn.Linear(5, 3)
data = autograd.Variable(torch.randn(2, 5))
print(lin(data))

Variable containing:
 0.3130  0.2576  1.3546
 1.0007  0.6433  0.4951
[torch.FloatTensor of size 2x3]



In [3]:
data = autograd.Variable(torch.randn(2, 2))
print(data)
print(F.relu(data))

Variable containing:
 1.2182  0.2117
-1.0613 -1.9441
[torch.FloatTensor of size 2x2]

Variable containing:
 1.2182  0.2117
 0.0000  0.0000
[torch.FloatTensor of size 2x2]



In [4]:
data = autograd.Variable(torch.randn(5))
print(data)
print(F.softmax(data))
print(F.softmax(data).sum())
print(F.log_softmax(data))

Variable containing:
-0.9596
 0.5489
-0.9901
-0.3826
 1.5037
[torch.FloatTensor of size 5]

Variable containing:
 0.0500
 0.2258
 0.0485
 0.0890
 0.5868
[torch.FloatTensor of size 5]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
-2.9964
-1.4880
-3.0269
-2.4195
-0.5331
[torch.FloatTensor of size 5]



In [5]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

In [6]:
test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

In [8]:
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

class BoWClassifier(nn.Module):
    
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        
        self.linear = nn.Linear(vocab_size, num_labels)
    
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec))
    
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

for param in model.parameters():
    print(param)

sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}
Parameter containing:

Columns 0 to 9 
-0.1184  0.1092  0.1180  0.0847  0.1837  0.1188 -0.0732 -0.1597  0.0754  0.0071
 0.1796 -0.0346  0.0130 -0.1186  0.0753 -0.0825 -0.0724 -0.1404  0.0732  0.1111

Columns 10 to 19 
 0.1476  0.1432  0.1548  0.1291 -0.1628  0.1293 -0.1808 -0.0890 -0.1295 -0.1729
 0.1313 -0.0343 -0.1889 -0.1827  0.0981  0.0486  0.1917  0.0630  0.0973 -0.0790

Columns 20 to 25 
 0.1483  0.0669 -0.1575  0.0365 -0.0309  0.0673
-0.0861 -0.0211  0.1135 -0.1090 -0.1556 -0.1673
[torch.FloatTensor of size 2x26]

Parameter containing:
1.00000e-02 *
 -2.0438
 -1.2066
[torch.FloatTensor of size 2]

Variable containing:
-0.4912 -0.9465
[torch.FloatTensor of size 1x2]



In [10]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [11]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# Print the matrix column corresponding to "creo"
print(next(model.parameters())[:, word_to_ix["creo"]])

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.
for epoch in range(100):
    for instance, label in data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Variable as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to SPANISH
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))

        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)

# Index corresponding to Spanish goes up, English goes down!
print(next(model.parameters())[:, word_to_ix["creo"]])

Variable containing:
-0.4910 -0.9467
[torch.FloatTensor of size 1x2]

Variable containing:
-0.8985 -0.5229
[torch.FloatTensor of size 1x2]

Variable containing:
 0.1476
 0.1313
[torch.FloatTensor of size 2]

Variable containing:
-0.1126 -2.2399
[torch.FloatTensor of size 1x2]

Variable containing:
-2.8425 -0.0600
[torch.FloatTensor of size 1x2]

Variable containing:
 0.5315
-0.2526
[torch.FloatTensor of size 2]

